# SNP Detection

## Setting the dependencies

In [23]:
snpFile   = '/home/senne/nanopore/SNP/known_SNP_sequence/SNP_sequence.fasta'  
readFile  = '/home/senne/nanopore/SNP/Nanopore_data/2_potential_snp_amplicons_3mism.fasta'
resultDir = '/home/senne/nanopore/SNP/results_yannick'
fastqFile = '/home/senne/nanopore/SNP/Nanopore_data/ligatedSNPs.fastq'
bwa       = '/opt/tools/bwa-0.7.15'     # v0.7.5
samtools  = '/opt/tools/samtools-1.3.1' # v1.3.1
bcftools  = '/opt/tools/bcftools-1.3.1' # v1.3.1

# Check
!ls {snpFile}
print('Number of SNPs in reference file:')
!grep -c ">" {snpFile}


/home/senne/nanopore/SNP/known_SNP_sequence/SNP_sequence.fasta
Number of SNPs in reference file:
52


## Info on the fastq file

In [24]:
import numpy as np

lengthData = []

with open(fastqFile, 'r') as inFile:
  lineCount = 0
  
  for line in inFile:
    lineCount += 1
  
    if lineCount % 4 == 2:
      lengthData.append(len(line.rstrip()))

# Display info
len (lengthData), np.mean(lengthData), np.median(lengthData) , max(lengthData), np.std(lengthData)


(14324, 507.74413571628037, 411.0, 5850, 324.59961624575982)

## Map reads to reference sequences

In [26]:
# Map reads to reference sequences

# Build index of the references
!{bwa} index {snpFile}

# Map reads
!{bwa} mem -a {snpFile} {readFile} > {resultDir}/SNPforID.sam

# Make sorted bam and index
!{samtools} view -Sbu {resultDir}/SNPforID.sam | {samtools} sort -o {resultDir}/SNPforID_sorted.bam -
!{samtools} index {resultDir}/SNPforID_sorted.bam {resultDir}/SNPforID_sorted.bam.bai
print('Done')


[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.15-r1140
[main] CMD: /opt/tools/bwa-0.7.15 index /home/senne/nanopore/SNP/known_SNP_sequence/SNP_sequence.fasta
[main] Real time: 0.016 sec; CPU: 0.013 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 23057 sequences (1988758 bp)...
[M::mem_process_seqs] Processed 23057 reads in 1.191 CPU sec, 1.189 real sec
[main] Version: 0.7.15-r1140
[main] CMD: /opt/tools/bwa-0.7.15 mem -a /home/senne/nanopore/SNP/known_SNP_sequence/SNP_sequence.fasta /home/senne/nanopore/SNP/Nanopore_data/2_potential_snp_amplicons_3mism.fasta
[main] Real time: 1.255 sec; CPU: 1.252 sec
Done


In [27]:
# Display some stats

!{samtools} flagstat {resultDir}/SNPforID_sorted.bam


23061 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
4 + 0 supplementary
0 + 0 duplicates
7824 + 0 mapped (33.93% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


## Generate vcf file

In [28]:
# Generate vcf file from bam file. Needs the reference and its index file 

# Note: the commands below are for samtools and bcftools v1.3.1 and will not work on v0.1.19!

# Reporting all positions
!{samtools} mpileup -d 100000 -uf {snpFile} {resultDir}/SNPforID_sorted.bam | {bcftools} call -V indels -m - > {resultDir}/SNPforID_sorted.bam.vcf

print('Done')

Note: Neither --ploidy nor --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
Done


## Generate the SNP profile 

In [29]:
# Get SNP profile

snpData = {}

with open(resultDir+'/SNPforID_sorted.bam.vcf') as f:
    for l in f:
        if l.startswith('#'):
            continue
            
        snp, pos, id, ref, alt, qual, filter, info, d, dd = l.split()
        
        # Our SNP of interest is always at position 26 of the reference
        if int(pos) != 26:
            continue

        par = {}
        for p in info.split(';'):
            pv = p.split('=')
            par[pv[0]] = pv[1]
        
        snpData[snp] = {'pos': pos, 'ref': ref, 'alt': alt, 'qual': qual, 'filter': filter, 'info': par}

# Display count
print('Got data for {} SNPs:'.format(len(snpData)))

# Save/print results
with open(resultDir + '/ttt_profile.csv', 'w') as f:
    # Table header
    f.write('snp, coverage, ref_allele, ref_percent, alt_allele, alt_percent, genotype\n')
    
    # Table data
    for s in sorted(snpData.keys()):
        totalDepth = int(snpData[s]['info']['DP'])
        depthList  = [int(d) for d in snpData[s]['info']['DP4'].split(',')]
        refDepth   = sum(depthList[0:2])
        altDepth   = sum(depthList[2:4])
        
        # Estimate the diploid genotype: when the minor allele is more than 10 times weaker than the major allele,
        # we should ignore it for a pure sample?
        if refDepth > altDepth and altDepth/refDepth < 0.1:
            genotype = snpData[s]['ref'] + snpData[s]['ref']
        elif altDepth > refDepth and refDepth/altDepth < 0.1:
            genotype = snpData[s]['alt'] + snpData[s]['alt']
        else:
            genotype = snpData[s]['ref'] + snpData[s]['alt']
        
        if snpData[s]['alt'] == '.':
            # Only 1 allele was observed
            f.write(','.join([s, str(totalDepth), snpData[s]['ref'], '{:.1f}'.format(100*refDepth/totalDepth), '', '', snpData[s]['ref']+snpData[s]['ref']]) + '\n')
            print('  {} ({})  {} ({:.1f} %)'.format(s, totalDepth, snpData[s]['ref'], 100*refDepth/totalDepth))
        else:
            # Two alleles were observed
            f.write(','.join([s, str(totalDepth), snpData[s]['ref'], '{:.1f}'.format(100*refDepth/totalDepth), snpData[s]['alt'], '{:.1f}'.format(100*altDepth/totalDepth), genotype]) + '\n')
            print('  {} ({})  {} ({:.1f} %)  {} ({:.1f} %)'.format(s, totalDepth, snpData[s]['ref'], 100*refDepth/totalDepth, snpData[s]['alt'], 100*altDepth/totalDepth))


Got data for 52 SNPs:
  rs1005533 (97)  A (62.9 %)  G (37.1 %)
  rs1015250 (63)  C (3.2 %)  G (96.8 %)
  rs1024116 (98)  A (40.8 %)  G (59.2 %)
  rs1028528 (134)  A (3.0 %)  G (97.0 %)
  rs1029047 (29)  A (100.0 %)
  rs1031825 (76)  A (26.3 %)  C (73.7 %)
  rs10495407 (177)  A (60.5 %)  G (39.5 %)
  rs1335873 (119)  A (1.7 %)  T (98.3 %)
  rs1355366 (158)  A (5.1 %)  G (94.9 %)
  rs1357617 (218)  A (100.0 %)
  rs1360288 (183)  C (60.1 %)  T (39.9 %)
  rs1382387 (200)  G (2.5 %)  T (97.5 %)
  rs1413212 (164)  A (65.2 %)  G (34.8 %)
  rs1454361 (195)  A (100.0 %)
  rs1463729 (148)  A (97.3 %)
  rs1490413 (224)  A (98.2 %)
  rs1493232 (69)  A (73.9 %)  C (26.1 %)
  rs1528460 (83)  C (2.4 %)  T (97.6 %)
  rs1886510 (113)  C (6.2 %)  T (93.8 %)
  rs1979255 (301)  C (98.0 %)
  rs2016276 (69)  A (95.7 %)
  rs2040411 (76)  A (0.0 %)  G (100.0 %)
  rs2046361 (110)  A (3.6 %)  T (96.4 %)
  rs2056277 (199)  C (47.7 %)  T (52.3 %)
  rs2076848 (123)  A (73.2 %)  T (26.8 %)
  rs2107612 (48)  A (0.0 